In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

### To run this notebook, first run all the pipelines with each of the example yaml files, using the same output_dir for all of them. 

In [ ]:
save = True #whether to save plots from notebook
output_dir = '/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_90_150' #output_dir that was used in the yaml files

# Power Spectrum Parameter Dependence

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(7,7), sharey=True, sharex=True, layout='constrained')
axs = axs.flatten()
b = 5 #bin
npoints = 15
start_val, end_val = [0.9**2, 1.1**2]
acmb_test_vals = np.linspace(start_val, end_val, npoints)
atsz_test_vals = np.linspace(start_val, end_val, npoints)
comps = ['CMB', 'ftSZ']
call_fit = lambda A_vec, expr: expr.subs('x0', A_vec[0]).subs('x1', A_vec[1])

ax = 0
for pipeline in range(3):
    #load file
    if pipeline == 0:
        fname = f'{output_dir}/data_vecs/best_fits_NILC_nongaussiantsz_2ksims_tszamp150_gaussianlkl_4scales.p'
    elif pipeline == 1:
        fname = f'{output_dir}/data_vecs/best_fits_HILC_nongaussiantsz_weightsvary_2ksims_tszamp150_gaussianlkl_sr.p'
    else:
        fname = f'{output_dir}/data_vecs/best_fits_HILC_nongaussiantsz_weightsonce_2ksims_tszamp150_gaussianlkl_sr.p'
    best_fits = pickle.load(open(fname, 'rb'))   
    
    for p,q in [(0,0), (0,1), (1,1)]:
        plt.axes(axs[ax])
        arr = np.zeros((npoints, npoints))
        for i,acmb in enumerate(acmb_test_vals):
            for j,atsz in enumerate(atsz_test_vals):
                arr[j,i] = call_fit([acmb, atsz], best_fits[p][q][b])

        plt.imshow(arr, extent=[start_val, end_val, start_val, end_val,], vmin=0.85, vmax=1.15, origin='lower')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        if ax >= 6:
            plt.xlabel(r'$A_{\mathrm{CMB}}$', fontsize=14)
        if ax % 3 == 0:
            plt.ylabel(r'$A_{\mathrm{ftSZ}}$', fontsize=14)
        ax += 1
        if pipeline == 0:
            title = 'NILC'
        elif pipeline == 1:
            title = 'HILC (weights vary)'
        else:
            title = 'HILC (weights fixed)'
        if (p,q) == (0,0):
            title += r' $C_\ell^{\hat{T}\hat{T}}$'
        elif (p,q) == (0,1):
            title += r' $C_\ell^{\hat{T}\hat{y}}$'
        else:
            title += r' $C_\ell^{\hat{y}\hat{y}}$'
        plt.title(title, fontsize=12)
    pipeline_str = 'NILC' if pipeline == 0 else 'HILC'
    if pipeline == 1:
        pipeline_str += ' weights vary'
    elif pipeline == 2:
        pipeline_str += ' weights once'
    print(f'{pipeline_str} expression for ClTT parameter dependence in bin {b}: ', best_fits[0][0][b])
    print(f'{pipeline_str} expression for ClTy parameter dependence in bin {b}: ', best_fits[0][1][b])
    print(f'{pipeline_str} expression for Clyy parameter dependence in bin {b}: ', best_fits[1][1][b])
    print()

handles, labels = axs[-1].get_legend_handles_labels() 
fig.legend(handles, labels, fontsize=10, bbox_to_anchor=(1.0, 0.05), ncol=2)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=12)

if save:
    plt.savefig('SR.pdf')

